We make sure to have the necessary units installed

In [ ]:
%pip install -r requirements.txt

## Initialization of a data source

In this example we will instantize the desired implementation of data source class

Different implementations are available to perform this task, based on obtaining the web pages of the respective package managers and also from CSV text files

### Data obtaining via web scarping

In the first place we import the implementation of the data source we want, for this example we will use the Bioconductor Scraper

In [2]:
from olivia_finder.scraping.bioconductor import BiocScraper

In [3]:
data_source = BiocScraper()

Show relevant information about the data source

In [4]:
print(data_source.get_info())

Name: Bioconductor
Description: Scraper class implementation for the Bioconductor package network


Get a list with the name of the packages obtained from this source

In [5]:
package_list = data_source.obtain_package_names()
package_list[:10]

['ABSSeq',
 'ABarray',
 'ACE',
 'ACME',
 'ADAM',
 'ADAMgui',
 'ADImpute',
 'ADaCGH2',
 'AGDEX',
 'AIMS']

We can obtain the data from a specific package, for example the **`DeepBlueR`** package

In [6]:
from olivia_finder.util.logger import UtilLogger
UtilLogger.init_logger()
UtilLogger.enable_logger()

In [7]:
deepbluer = data_source.obtain_package_data("DeepBlueR")
deepbluer

2023-03-13 21:18:59 [   DEBUG] Scraping package DeepBlueR (logger.py:96)
2023-03-13 21:18:59 [   DEBUG] Getting next proxy (logger.py:96)
2023-03-13 21:18:59 [   DEBUG] Proxy list rotated, using 202.29.218.138:4153, next will be 185.162.230.201:80 (logger.py:96)
2023-03-13 21:18:59 [   DEBUG] Using proxy: {'http': 'http://202.29.218.138:4153'} (logger.py:96)
2023-03-13 21:18:59 [   DEBUG] Getting next useragent (logger.py:96)
2023-03-13 21:18:59 [   DEBUG] Using user agent: Mozilla/5.0 (Linux; Android 4.4.2; RCT6203W46 Build/KOT49H) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.84 Safari/537.36 (logger.py:96)
2023-03-13 21:18:59 [   DEBUG] Starting new HTTPS connection (1): www.bioconductor.org:443 (connectionpool.py:1003)
2023-03-13 21:18:59 [   DEBUG] https://www.bioconductor.org:443 "GET /packages/release/bioc/html/DeepBlueR.html HTTP/1.1" 200 5671 (connectionpool.py:456)
2023-03-13 21:18:59 [   DEBUG] Response status code: 200 (logger.py:96)
2023-03-13 21:18:59 [   DEBUG]

{'name': 'DeepBlueR',
 'version': '1.24.1',
 'dependencies': [{'name': 'R', 'version': '>= 3.3'},
  {'name': 'XML', 'version': ''},
  {'name': 'RCurl', 'version': ''},
  {'name': 'GenomicRanges', 'version': ''},
  {'name': 'data.table', 'version': ''},
  {'name': 'stringr', 'version': ''},
  {'name': 'diffr', 'version': ''},
  {'name': 'dplyr', 'version': ''},
  {'name': 'methods', 'version': ''},
  {'name': 'rjson', 'version': ''},
  {'name': 'utils', 'version': ''},
  {'name': 'R.utils', 'version': ''},
  {'name': 'foreach', 'version': ''},
  {'name': 'withr', 'version': ''},
  {'name': 'rtracklayer', 'version': ''},
  {'name': 'GenomeInfoDb', 'version': ''},
  {'name': 'settings', 'version': ''},
  {'name': 'filehash', 'version': ''}],
 'url': 'https://www.bioconductor.org/packages/release/bioc/html/DeepBlueR.html'}

Be careful with the sensitivity to **caps**, if the package has not been found, an **ScraperError** exception is raised

In [8]:
try:
    deepbluer2 = data_source.obtain_package_data("deepbluer")
except Exception as e:
    print(e)

2023-03-13 21:19:05 [   DEBUG] Scraping package deepbluer (logger.py:96)
2023-03-13 21:19:05 [   DEBUG] Getting next proxy (logger.py:96)
2023-03-13 21:19:05 [   DEBUG] Proxy list rotated, using 185.162.230.201:80, next will be 150.220.8.228:64312 (logger.py:96)
2023-03-13 21:19:05 [   DEBUG] Using proxy: {'http': 'http://185.162.230.201:80'} (logger.py:96)
2023-03-13 21:19:05 [   DEBUG] Getting next useragent (logger.py:96)
2023-03-13 21:19:05 [   DEBUG] Using user agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.101 Safari/537.36 (logger.py:96)
2023-03-13 21:19:05 [   DEBUG] Starting new HTTPS connection (1): www.bioconductor.org:443 (connectionpool.py:1003)
2023-03-13 21:19:06 [   DEBUG] https://www.bioconductor.org:443 "GET /packages/release/bioc/html/deepbluer.html HTTP/1.1" 404 6873 (connectionpool.py:456)
2023-03-13 21:19:06 [   DEBUG] Response status code: 404 (logger.py:96)
2023-03-13 21:19:06 [   DEBUG] ScraperError: Package d

ScraperError: Package deepbluer not found
super


In [9]:
UtilLogger.disable_logger()
pkgs_data = data_source.obtain_packages_data(package_list[:3])
pkgs_data

[{'name': 'ABSSeq',
  'version': '1.52.0',
  'dependencies': [{'name': 'R', 'version': '>= 2.10'},
   {'name': 'methods', 'version': ''},
   {'name': 'locfit', 'version': ''},
   {'name': 'limma', 'version': ''}],
  'url': 'https://www.bioconductor.org/packages/release/bioc/html/ABSSeq.html'},
 {'name': 'ABarray',
  'version': '1.66.0',
  'dependencies': [{'name': 'Biobase', 'version': ''},
   {'name': 'graphics', 'version': ''},
   {'name': 'grDevices', 'version': ''},
   {'name': 'methods', 'version': ''},
   {'name': 'multtest', 'version': ''},
   {'name': 'stats', 'version': ''},
   {'name': 'tcltk', 'version': ''},
   {'name': 'utils', 'version': ''}],
  'url': 'https://www.bioconductor.org/packages/release/bioc/html/ABarray.html'},
 {'name': 'ACE',
  'version': '1.16.0',
  'dependencies': [{'name': 'R', 'version': '>= 3.4'},
   {'name': 'Biobase', 'version': ''},
   {'name': 'QDNAseq', 'version': ''},
   {'name': 'ggplot2', 'version': ''},
   {'name': 'grid', 'version': ''},
   {

In [10]:
for p in pkgs_data:
    print(f'Package: {p["name"]} ({p["version"]})')

    for d in p["dependencies"]:
        print(f'-   Dependency: {d["name"]} {d["version"]}')

Package: ABSSeq (1.52.0)
-   Dependency: R >= 2.10
-   Dependency: methods 
-   Dependency: locfit 
-   Dependency: limma 
Package: ABarray (1.66.0)
-   Dependency: Biobase 
-   Dependency: graphics 
-   Dependency: grDevices 
-   Dependency: methods 
-   Dependency: multtest 
-   Dependency: stats 
-   Dependency: tcltk 
-   Dependency: utils 
Package: ACE (1.16.0)
-   Dependency: R >= 3.4
-   Dependency: Biobase 
-   Dependency: QDNAseq 
-   Dependency: ggplot2 
-   Dependency: grid 
-   Dependency: stats 
-   Dependency: utils 
-   Dependency: methods 
-   Dependency: grDevices 
-   Dependency: GenomicRanges 


### Data obtaining from CSV files

In [1]:
from olivia_finder.csv_network import CSVNetwork

In [2]:
# Load the network
csv_data_source = CSVNetwork("Bioconductor", "Bioconductor as a CSV file")
csv_data_source.load_data(
    file_path="results/csv_datasets/bioconductor_adjlist_scraping.csv", 
    dependent_field="name", dependency_field="dependency"
)

In [3]:
package_list = csv_data_source.obtain_package_names()
package_list[:10]

['ABSSeq',
 'ABarray',
 'ACE',
 'ACME',
 'ADAM',
 'ADAMgui',
 'ADImpute',
 'ADaCGH2',
 'AGDEX',
 'AIMS']

In [4]:
deepbluer = csv_data_source.obtain_package_data("DeepBlueR")
deepbluer

{'name': 'DeepBlueR',
 'version': '1.24.1',
 'url': 'https://www.bioconductor.org/packages/release/bioc/html/DeepBlueR.html',
 'dependencies': [{'name': 'R', 'version': '>= 3.3'},
  {'name': 'XML', 'version': nan},
  {'name': 'RCurl', 'version': nan},
  {'name': 'GenomicRanges', 'version': nan},
  {'name': 'data.table', 'version': nan},
  {'name': 'stringr', 'version': nan},
  {'name': 'diffr', 'version': nan},
  {'name': 'dplyr', 'version': nan},
  {'name': 'methods', 'version': nan},
  {'name': 'rjson', 'version': nan},
  {'name': 'utils', 'version': nan},
  {'name': 'R.utils', 'version': nan},
  {'name': 'foreach', 'version': nan},
  {'name': 'withr', 'version': nan},
  {'name': 'rtracklayer', 'version': nan},
  {'name': 'GenomeInfoDb', 'version': nan},
  {'name': 'settings', 'version': nan},
  {'name': 'filehash', 'version': nan}]}

In [5]:
packages = csv_data_source.obtain_packages_data(package_list[:3])
packages

100%|██████████| 3/3 [00:00<00:00, 604.89it/s]


[{'name': 'ABSSeq',
  'version': '1.52.0',
  'url': 'https://www.bioconductor.org/packages/release/bioc/html/ABSSeq.html',
  'dependencies': [{'name': 'R', 'version': '>= 2.10'},
   {'name': 'methods', 'version': nan},
   {'name': 'locfit', 'version': nan},
   {'name': 'limma', 'version': nan}]},
 {'name': 'ABarray',
  'version': '1.66.0',
  'url': 'https://www.bioconductor.org/packages/release/bioc/html/ABarray.html',
  'dependencies': [{'name': 'Biobase', 'version': nan},
   {'name': 'graphics', 'version': nan},
   {'name': 'grDevices', 'version': nan},
   {'name': 'methods', 'version': nan},
   {'name': 'multtest', 'version': nan},
   {'name': 'stats', 'version': nan},
   {'name': 'tcltk', 'version': nan},
   {'name': 'utils', 'version': nan}]},
 {'name': 'ACE',
  'version': '1.16.0',
  'url': 'https://www.bioconductor.org/packages/release/bioc/html/ACE.html',
  'dependencies': [{'name': 'R', 'version': '>= 3.4'},
   {'name': 'Biobase', 'version': nan},
   {'name': 'QDNAseq', 'versi

## Initialization of a package manager

In [1]:
from olivia_finder.package_manager import PackageManager
from olivia_finder.util.logger import UtilLogger

### Web scraping implementation

In [2]:
from olivia_finder.scraping.pypi import PypiScraper

UtilLogger.enable_logger()
pypi = PackageManager(PypiScraper())

2023-03-13 22:10:35 [   DEBUG] Added SSLProxies to proxy builders (logger.py:96)
2023-03-13 22:10:35 [   DEBUG] Added FreeProxyList to proxy builders (logger.py:96)
2023-03-13 22:10:35 [   DEBUG] Added GeonodeProxy to proxy builders (logger.py:96)
2023-03-13 22:10:35 [   DEBUG] Starting new HTTPS connection (1): www.sslproxies.org:443 (connectionpool.py:1003)
2023-03-13 22:10:35 [   DEBUG] https://www.sslproxies.org:443 "GET / HTTP/1.1" 200 None (connectionpool.py:456)
2023-03-13 22:10:35 [   DEBUG] Found 100 proxies from SSLProxies (logger.py:96)
2023-03-13 22:10:35 [   DEBUG] Starting new HTTPS connection (1): free-proxy-list.net:443 (connectionpool.py:1003)
2023-03-13 22:10:35 [   DEBUG] https://free-proxy-list.net:443 "GET /anonymous-proxy.html HTTP/1.1" 200 None (connectionpool.py:456)
2023-03-13 22:10:36 [   DEBUG] Found 100 proxies from FreeProxyList (logger.py:96)
2023-03-13 22:10:36 [   DEBUG] Starting new HTTPS connection (1): proxylist.geonode.com:443 (connectionpool.py:1003

In [3]:
networkx = pypi.obtain_package("networkx")
networkx.print()

2023-03-13 22:10:43 [   DEBUG] Scraping package networkx (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Getting next proxy (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Proxy list rotated, using 34.87.103.220:80, next will be 95.107.183.18:5678 (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Using proxy: {'http': 'http://34.87.103.220:80'} (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Getting next useragent (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Using user agent: Mozilla/5.0 (iPad; CPU OS 8_1_3 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12B466 Safari/600.1.4 (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Starting new HTTPS connection (1): pypi.org:443 (connectionpool.py:1003)
2023-03-13 22:10:43 [   DEBUG] https://pypi.org:443 "GET /pypi/networkx/json HTTP/1.1" 200 35312 (connectionpool.py:456)
2023-03-13 22:10:43 [   DEBUG] Response status code: 200 (logger.py:96)
2023-03-13 22:10:43 [   DEBUG] Package networkx scraped successfully (logger.py:96)


Package:
  name: networkx
  version: 3.0
  url: https://pypi.org/project/networkx/
  dependencies:
    numpy:(>=1.20)
    scipy:(>=1.8)
    matplotlib:(>=3.4)
    pandas:(>=1.3)
    pre-commit:(>=2.20)
    mypy:(>=0.991)
    sphinx:(==5.2.3)
    pydata-sphinx-theme:(>=0.11)
    sphinx-gallery:(>=0.11)
    numpydoc:(>=1.5)
    pillow:(>=9.2)
    nb2plots:(>=0.6)
    texext:(>=0.6.7)
    lxml:(>=4.6)
    pygraphviz:(>=1.10)
    pydot:(>=1.4.2)
    sympy:(>=1.10)
    pytest:(>=7.2)
    pytest-cov:(>=4.0)
    codecov:(>=2.1)


In [4]:
UtilLogger.disable_logger()
packages = pypi.obtain_packages(["networkx", "numpy", "pandas"])
packages

In [5]:
packages[2].print()

Package:
  name: pandas
  version: 1.5.3
  url: https://pypi.org/project/pandas/
  dependencies:
    python-dateutil:(>=2.8.1)
    pytz:(>=2020.1)
    numpy:(>=1.23.2)
    hypothesis:(>=5.5.3)
    pytest:(>=6.0)
    pytest-xdist:(>=1.31)


Getting all the packages from a package manager can take a while, so it is recommended to save the data to a CSV file for later use.

We can see that the execution time for half a million packages (Pypi) is around 7 hours.
In the case of Bioconductor, to obtain the 2000 packages it contains, the execution time is around 4 minutes.

In [7]:
pypi_packages = pypi.obtain_packages(extend=True, show_progress=True)

 15%|█▌        | 66423/438514 [1:14:50<6:40:39, 15.48it/s] 

In [ ]:
bioconductor = PackageManager(BiocScraper())
bioconductor_packages = bioconductor.obtain_packages(extend=True, show_progress=True)

100%|██████████| 2183/2183 [03:47<00:00,  9.59it/s]


In [ ]:
import pickle
# Save the bioconductor package manager as a pickle file
with open("./results/package_managers/bioconductor_pm_scraping.pkl", "wb") as f:
    pickle.dump(bioconductor_packages, f)

In [ ]:
# Store the package manager as a adjacency list
b_df = bioconductor.export_full_adjlist()

# Store the package manager as a adjacency list
b_df.to_csv("./results/csv_datasets/bioconductor_adjlist_scraping.csv", index=False)


In [ ]:
# Get the package with the most dependencies
max_deps = max(bioconductor_packages, key=lambda p: len(p.dependencies))
max_deps.print()
print(f'Number of dependencies: {len(max_deps.dependencies)}')

Package:
  name: singleCellTK
  version: 2.8.0
  url: https://www.bioconductor.org/packages/release/bioc/html/singleCellTK.html
  dependencies:
    R:>= 4.0
    SummarizedExperiment:
    SingleCellExperiment:
    DelayedArray:
    Biobase:
    ape:
    AnnotationHub:
    batchelor:
    BiocParallel:
    celldex:
    colourpicker:
    colorspace:
    cowplot:
    cluster:
    ComplexHeatmap:
    data.table:
    DelayedMatrixStats:
    DESeq2:
    dplyr:
    DT:
    ExperimentHub:
    ensembldb:
    fields:
    ggplot2:
    ggplotify:
    ggrepel:
    ggtree:
    gridExtra:
    GSVA:
    GSVAdata:
    igraph:
    KernSmooth:
    limma:
    MAST:
    Matrix:
    matrixStats:
    methods:
    msigdbr:
    multtest:
    plotly:
    plyr:
    ROCR:
    Rtsne:
    S4Vectors:
    scater:
    scMerge:
    scran:
    Seurat:>= 3.1.3
    shiny:
    shinyjs:
    SingleR:
    SoupX:
    sva:
    reshape2:
    shinyalert:
    circlize:
    enrichR:
    celda:
    shinycssloaders:
    DropletUtils:
 